In [ ]:
import sys
sys.path.append("..")

In [ ]:
from emeval.input.spec_details import FileSpecDetails

SPEC_ID = "train_bus_ebike_mtv_ucb"
sd = FileSpecDetails("data/", "shankari@eecs.berkeley.edu", SPEC_ID)

In [ ]:
from emeval.input.phone_view import PhoneView

pv = PhoneView(sd)

In [ ]:
import pandas as pd

df_map = dict()

for phone_os, phones in pv.map().items():
    for phone_label, phone_map in phones.items():
        pk = {"phone_label": phone_label, "role": phone_map["role"]}
        for r in (phone_map["calibration_ranges"] + phone_map["evaluation_ranges"]):
            for df_label, df in {k: v for k, v in r.items() if isinstance(v, pd.DataFrame)}.items():
                curr_pk = pk.copy()
                
                if r in phone_map["calibration_ranges"]:
                    curr_pk.update({"range_type": "calibration"})
                elif r in phone_map["evaluation_ranges"]:
                    curr_pk.update({"range_type": "evaluation"})
                
                curr_pk.update({k: v for k, v in r.items()
                                if not (
                                    isinstance(v, pd.DataFrame)\
                                    or isinstance(v, list)\
                                    or isinstance(v, dict))})
                
                updated_df = df.assign(**curr_pk)
                
                for ts_col in [c for c in updated_df.columns.tolist() if "ts" in c]:
                    updated_df[ts_col] = updated_df[ts_col].astype(float)

                if phone_os not in df_map:
                    df_map[phone_os] = dict()
                
                if df_label not in df_map[phone_os]:
                    df_map[phone_os][df_label] = updated_df
                else:
                    df_map[phone_os][df_label] = pd.concat([df_map[phone_os][df_label], updated_df], ignore_index=True)

In [ ]:
from IPython.display import display

for phone_os, df_detail_map in df_map.items():
    for df_label, df in df_detail_map.items():
        print(phone_os, df_label)
        display(df)